In [3]:
%config Completer.use_jedi = False
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

UsageError: Line magic function `%%capture` not found.


In [31]:
%%capture
%load_ext autoreload
%autoreload 2

import warnings
import numpy as np
import numpy.matlib as npm
import scipy.io as sio

from keras.utils.np_utils import to_categorical

from bcilib.ssvep_utils_pytorch_test import CNN, RasterizeSlice, CustomTensorDataset
from bcilib import ssvep_utils as su 
from torch.utils.data import TensorDataset, DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
import torch
import tqdm

warnings.filterwarnings('ignore')

In [32]:
input_shape = (8, 220, 1)

from torchsummary import summary
summary(
    CNN().network.cuda(),
    input_shape
)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 213, 1]          2,048
├─BatchNorm2d: 1-2                       [-1, 32, 213, 1]          64
├─ReLU: 1-3                              [-1, 32, 213, 1]          --
├─Dropout: 1-4                           [-1, 32, 213, 1]          --
├─Conv2d: 1-5                            [-1, 16, 204, 1]          5,120
├─BatchNorm2d: 1-6                       [-1, 16, 204, 1]          32
├─ReLU: 1-7                              [-1, 16, 204, 1]          --
├─Dropout: 1-8                           [-1, 16, 204, 1]          --
├─Flatten: 1-9                           [-1, 3264]                --
├─Linear: 1-10                           [-1, 12]                  39,168
Total params: 46,432
Trainable params: 46,432
Non-trainable params: 0
Total mult-adds (M): 1.52
Input size (MB): 0.01
Forward/backward pass size (MB): 0.15
Params size (MB): 0.18
Estimated Total Size (MB): 0.34


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 213, 1]          2,048
├─BatchNorm2d: 1-2                       [-1, 32, 213, 1]          64
├─ReLU: 1-3                              [-1, 32, 213, 1]          --
├─Dropout: 1-4                           [-1, 32, 213, 1]          --
├─Conv2d: 1-5                            [-1, 16, 204, 1]          5,120
├─BatchNorm2d: 1-6                       [-1, 16, 204, 1]          32
├─ReLU: 1-7                              [-1, 16, 204, 1]          --
├─Dropout: 1-8                           [-1, 16, 204, 1]          --
├─Flatten: 1-9                           [-1, 3264]                --
├─Linear: 1-10                           [-1, 12]                  39,168
Total params: 46,432
Trainable params: 46,432
Non-trainable params: 0
Total mult-adds (M): 1.52
Input size (MB): 0.01
Forward/backward pass size (MB): 0.15
Params size (MB): 0.18
Estimated Total Size (MB): 0.34

In [10]:
def get_training_data(features_data):
    features_data = np.reshape(
        features_data,
        (
            features_data.shape[0],
            features_data.shape[1],
            features_data.shape[2],
            features_data.shape[3] * features_data.shape[4]
        )
    )
    train_data = features_data[:, :, 0, :].T
    for target in range(1, features_data.shape[2]):
        train_data = np.vstack(
            [train_data, np.squeeze(features_data[:, :, target, :]).T]
        )

    train_data = np.reshape(train_data, (train_data.shape[0], train_data.shape[1], 
                                         train_data.shape[2], 1))
    total_epochs_per_class = features_data.shape[3]
    features_data = []
    class_labels = np.arange(CNN_PARAMS['num_classes'])
    labels = (npm.repmat(class_labels, total_epochs_per_class, 1).T).ravel()
    labels = to_categorical(labels)
    
    return train_data, labels

In [11]:
def train_CNN_cross_val_predict(model, train_data, labels, num_folds=10):
    print("INPUT SHAPE: ", train_data.shape)
    
    labels = np.argmax(labels, axis=1)
    print("LABEL SHAPE: ", labels.shape)
    
    # frame_dataset = ShapesNpzDataset("/home/nogay/Desktop/frame_dataset/dataset.npz", target_transform=int)
    train_size = int(0.6 * len(train_data))
    val_size = int(0.2 * len(train_data))
    test_size = len(train_data) - train_size - val_size
    print(train_size, val_size, test_size)
    
    
    tensor_x = torch.from_numpy(train_data) # transform to torch tensor
    tensor_y = torch.from_numpy(labels.astype(int))
    
    print(tensor_x.dtype)
    print(tensor_y.dtype)

    dataset = TensorDataset(tensor_x, tensor_y) # create your datset
    
    train_dataset, val_dataset, test_dataset = random_split(
        dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    dataloader_train = DataLoader(train_dataset, shuffle=True, num_workers=4, batch_size=16)
    dataloader_val = DataLoader(val_dataset, shuffle=False, num_workers=4, batch_size=16)
    dataloader_test = DataLoader(test_dataset, shuffle=False, num_workers=4, batch_size=16)
    
    early_stopping = EarlyStopping('train_loss', patience=10, mode='min')
    trainer = pl.Trainer(gpus=1, 
                         max_epochs=200, 
                         enable_pl_optimizer=True,
                         callbacks=[early_stopping])
    
    trainer.fit(model, dataloader_train, val_dataloaders=dataloader_val)
    return trainer.test(model, test_dataloaders=dataloader_test, ckpt_path=None)
    #return cv_acc

In [16]:
data_path = os.path.abspath('data/third_change')

CNN_PARAMS = {
    'batch_size': 64,
    'epochs': 50,
    'droprate': 0.25,
    'learning_rate': 0.001,
    'lr_decay': 0.0,
    'l2_lambda': 0.0001,
    'momentum': 0.9,
    'kernel_f': 10,
    'n_ch': 8,
    'num_classes': 12}

FFT_PARAMS = {
    'resolution': 0.2930,
    'start_frequency': 3.0,
    'end_frequency': 35.0,
    'sampling_rate': 256
}

window_len = 1
shift_len = 1
    
all_acc = np.zeros((10, 1))

complex_spectrum_features = dict()
ccnn_training_data = dict()
ccnn_results = dict()

In [17]:
all_segmented_data = dict()
for subject in range(0, 10):
    dataset = sio.loadmat(f'{data_path}/s{subject+1}.mat')
    eeg = np.array(dataset['eeg'], dtype='float32')
    
    CNN_PARAMS['num_classes'] = eeg.shape[0]
    CNN_PARAMS['n_ch'] = eeg.shape[1]
    total_trial_len = eeg.shape[2]
    num_trials = eeg.shape[3]
    sample_rate = 256

    filtered_data = su.get_filtered_eeg(eeg, 6, 80, 4, sample_rate)
    all_segmented_data[f's{subject+1}'] = su.get_segmented_epochs(filtered_data, window_len, 
                                                                  shift_len, sample_rate)

In [20]:
for subject in all_segmented_data.keys():
    complex_spectrum_features[subject] = su.complex_spectrum_features(
        all_segmented_data[subject],
        FFT_PARAMS
    )

In [23]:
for subject in all_segmented_data.keys():
    ccnn_training_data[subject] = dict()
    train_data, labels = get_training_data(complex_spectrum_features[subject])
    
    # Data manipulation
    train_data = np.power(((train_data - train_data.min() + 1) / (train_data.max() - train_data.min())) * 100, 3)
    
    ccnn_training_data[subject]['train_data'] = train_data
    ccnn_training_data[subject]['label'] = labels

In [24]:
c_acc_list = []
model_list = []

for subject, i in zip(ccnn_training_data.keys(), range(len(ccnn_training_data))):
    print(f'\nCCNN - Subject: {subject}')
    train_data = ccnn_training_data[subject]['train_data']
    labels = ccnn_training_data[subject]['label']
    m = CNN()
    model_list.append(m)
    c_acc_list.append(train_CNN_cross_val_predict(m, train_data, labels, 10))
    torch.save(m, "models/cnn-s{}.h5".format(i))


GPU available: True, used: True
TPU available: False, using: 0 TPU cores

CCNN - Subject: s1
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 46.4 K
---------------------------------------
46.4 K    Trainable params
0         Non-trainable params
46.4 K    Total params
0.186     Total estimated model params size (MB)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 46.4 K
---------------------------------------
46.4 K    Trainable params
0         Non-trainable params
46.4 K    Total params
0.186     Total estimated model params size (MB)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.5138888888888888, 'test_loss': 1.4368135929107666}
--------------------------------------------------------------------------------

CCNN - Subject: s2
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 46.4 K
---------------------------------------
46.4 K    Trainable params
0         Non-trainable params
46.4 K    Total params
0.186     Total estimated model params size (MB)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.25, 'test_loss': 3.533494710922241}
--------------------------------------------------------------------------------

CCNN - Subject: s3
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 46.4 K
---------------------------------------
46.4 K    Trainable params
0         Non-trainable params
46.4 K    Total params
0.186     Total estimated model params size (MB)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9652777777777778, 'test_loss': 0.1460673213005066}
--------------------------------------------------------------------------------

CCNN - Subject: s9
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 46.4 K
---------------------------------------
46.4 K    Trainable params
0         Non-trainable params
46.4 K    Total params
0.186     Total estimated model params size (MB)

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9236111111111112, 'test_loss': 0.3057335913181305}
--------------------------------------------------------------------------------

CCNN - Subject: s10
INPUT SHAPE:  (720, 8, 220, 1)
LABEL SHAPE:  (720,)
432 144 144
torch.float64
torch.int64


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]




--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8194444444444444, 'test_loss': 0.5658348202705383}
--------------------------------------------------------------------------------


In [25]:
l = 0
for t in c_acc_list:
    print(t[0]["test_acc"])
    l += t[0]["test_acc"]
print("Overall ACC: ", l/len(c_acc_list))

0.5138888888888888
0.25
0.8263888888888888
0.9652777777777778
0.9722222222222222
0.9930555555555556
0.8333333333333334
0.9652777777777778
0.9236111111111112
0.8194444444444444
Overall ACC:  0.80625
